<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/tensorrt_torchtrt_efficientnet/nvidia_logo.png" width="90px">

# PySpark LLM Inference: DeepSeek-R1

In this notebook, we demonstrate distributed batch inference with [DeepSeek-R1](https://github.com/deepseek-ai/DeepSeek-R1), using open weights on Huggingface.

We use [DeepSeek-R1-Distill-Llama-8B](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Llama-8B) as demonstration. DeepSeek's distilled models are based on open-source LLMs (such as Llama/Qwen), and are fine-tuned using samples generated by DeepSeek-R1 to perform multi-step reasoning tasks.

**Note:** Running this model on GPU with 16-bit precision requires **~18GB** of GPU RAM.

In [4]:
import os

# Check the cluster environment to handle any platform-specific configurations.
on_databricks = os.environ.get("DATABRICKS_RUNTIME_VERSION", False)
on_dataproc = os.environ.get("DATAPROC_IMAGE_VERSION", False)
on_standalone = not (on_databricks or on_dataproc)

In [5]:
# Set the huggingface cache dir to dbfs/gcs so that executors can load the model from cache.
if on_databricks:
    hf_home = "/dbfs/FileStore/hf-home"
    dbutils.fs.mkdirs(hf_home)
    os.environ["HF_HOME"] = hf_home
elif on_dataproc:
    hf_home = "/mnt/gcs/hf-home"
    os.mkdir(hf_home) if not os.path.exists(hf_home) else None
    os.environ["HF_HOME"] = hf_home

## Warmup: Running locally

**Note:** If the driver node does not have sufficient GPU capacity, proceed to the PySpark section.

In [3]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Llama-8B", torch_dtype=torch.bfloat16, device="cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [10]:
res = pipe(["How many r's are there in strawberry?"], max_new_tokens=512, temperature=0.1)
print("\n", res[0][0]['generated_text'])


 How many r's are there in strawberry? Let's count them.

First, I'll write down the word: S T R A W B E R R Y.

Now, I'll go through each letter one by one.

1. S - no R.
2. T - no R.
3. R - that's one R.
4. A - no R.
5. W - no R.
6. B - no R.
7. E - no R.
8. R - that's two R's.
9. R - that's three R's.
10. Y - no R.

So, in total, there are three R's in the word strawberry.
</think>

To determine how many **r's** are in the word **strawberry**, let's follow these steps:

1. **Write down the word:**
   
   S T R A W B E R R Y

2. **Identify and count each occurrence of the letter R:**
   
   - **1.** S - no R
   - **2.** T - no R
   - **3.** R - **1 R**
   - **4.** A - no R
   - **5.** W - no R
   - **6.** B - no R
   - **7.** E - no R
   - **8.** R - **2 R's**
   - **9.** R - **3 R's**
   - **10.** Y - no R

3. **Total count of R's:**
   
   There are **3 R's** in the word **strawberry**.

\boxed{3}


In [3]:
res = pipe(["Which number is bigger: 9.9 or 9.11?"], max_new_tokens=512, temperature=0.1)
print("\n", res[0][0]['generated_text'])


 Which number is bigger: 9.9 or 9.11? Let's see.

First, I need to compare the whole number parts of both numbers. Both 9.9 and 9.11 have the same whole number part, which is 9.

Since the whole numbers are equal, I'll compare the decimal parts. For 9.9, the decimal part is 0.9, and for 9.11, the decimal part is 0.11.

To make it easier, I can express 0.9 as 0.90. Now, comparing 0.90 and 0.11, it's clear that 0.90 is greater than 0.11.

Therefore, 9.9 is bigger than 9.11.
</think>

To determine which number is larger between **9.9** and **9.11**, let's compare them step by step.

1. **Compare the Whole Numbers:**
   - Both numbers have the same whole number part: **9**.
   
2. **Compare the Decimal Parts:**
   - **9.9** can be written as **9.90**.
   - **9.11** remains **9.11**.
   
3. **Analyze the Decimal Comparison:**
   - Compare the tenths place:
     - **9.90** has **9** in the tenths place.
     - **9.11** has **1** in the tenths place.
   - Since **9 > 1**, **9.90** is greater

In [4]:
import torch

# Unload the model from GPU memory.
del pipe
torch.cuda.empty_cache()

## PySpark

In [1]:
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, col, struct, length
from pyspark.ml.functions import predict_batch_udf

In [2]:
import os
import datasets
from datasets import load_dataset
datasets.disable_progress_bars()

#### Create Spark Session

For local standalone clusters, we'll connect to the cluster and create the Spark Session.  
For CSP environments, Spark will either be preconfigured (Databricks) or we'll need to create the Spark Session (Dataproc).

In [6]:
conf = SparkConf()

if 'spark' not in globals():
    if on_standalone:
        import socket
        conda_env = os.environ.get("CONDA_PREFIX")
        hostname = socket.gethostname()
        conf.setMaster(f"spark://{hostname}:7077")
        conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
        conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")
        # Point PyTriton to correct libpython3.11.so:
        conf.set("spark.executorEnv.LD_LIBRARY_PATH", f"{conda_env}/lib:{conda_env}/lib/python3.11/site-packages/nvidia_pytriton.libs:$LD_LIBRARY_PATH")
    elif on_dataproc:
        # Point PyTriton to correct libpython3.11.so:
        conda_lib_path="/opt/conda/miniconda3/lib"
        conf.set("spark.executorEnv.LD_LIBRARY_PATH", f"{conda_lib_path}:$LD_LIBRARY_PATH")
        conf.set("spark.executor.instances", "4") # dataproc defaults to 2

    conf.set("spark.executor.cores", "8")
    conf.set("spark.task.maxFailures", "1")
    conf.set("spark.task.resource.gpu.amount", "0.125")
    conf.set("spark.executor.resource.gpu.amount", "1")
    conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
    conf.set("spark.python.worker.reuse", "true")

conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1000")
spark = SparkSession.builder.appName("spark-dl-examples").config(conf=conf).getOrCreate()
sc = spark.sparkContext

25/02/09 20:39:35 WARN Utils: Your hostname, cb4ae00-lcedt resolves to a loopback address: 127.0.1.1; using 10.110.47.100 instead (on interface eno1)
25/02/09 20:39:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/09 20:39:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Load the Orca Math Word Problems dataset from Huggingface.

In [7]:
dataset = load_dataset("microsoft/orca-math-word-problems-200k", split="train[:1%]")
dataset = dataset.to_pandas()["question"]

#### Create PySpark DataFrame

In [8]:
df = spark.createDataFrame(dataset, schema=StringType()).withColumnRenamed("value", "question")
df.show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                            question|
+----------------------------------------------------------------------------------------------------+
|Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jung...|
|A number divided by 10 is 6. Yoongi got the result by subtracting 15 from a certain number. What ...|
|Dongju selects a piece of paper with a number written on it, and wants to make a three-digit numb...|
|You wanted to subtract 46 from a number, but you accidentally subtract 59 and get 43. How much do...|
|The length of one span of Jinseo is about 12 centimeters (cm). When Jinseo measured the length of...|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [9]:
df.write.mode("overwrite").json("spark-dl-datasets/orca_math")

## Triton Inference Server
We'll demonstrate integration with the [Triton Inference Server](https://developer.nvidia.com/nvidia-triton-inference-server), an open-source, GPU-accelerated serving solution for DL.  
We use [PyTriton](https://github.com/triton-inference-server/pytriton), a Flask-like framework that handles client/server communication with the Triton server.  

The process looks like this:
- Distribute a PyTriton task across the Spark cluster, instructing each node to launch a Triton server process.
- Define a Triton inference function, which contains a client that binds to the local server on a given node and sends inference requests.
- Wrap the Triton inference function in a predict_batch_udf to launch parallel inference requests using Spark.
- Finally, distribute a shutdown signal to terminate the Triton server processes on each node.

<img src="../images/spark-pytriton.png" alt="drawing" width="700"/>

In [18]:
from functools import partial

Import the utility functions from pytriton_utils.py:

In [ ]:
sc.addPyFile("pytriton_utils.py")

from pytriton_utils import TritonServerManager

Define the Triton Server function:

In [20]:
def triton_server(ports):
    import time
    import signal
    import numpy as np
    import torch
    from transformers import pipeline
    from pytriton.decorators import batch
    from pytriton.model_config import DynamicBatcher, ModelConfig, Tensor
    from pytriton.triton import Triton, TritonConfig
    from pyspark import TaskContext

    print(f"SERVER: Initializing model on worker {TaskContext.get().partitionId()}.")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Llama-8B", torch_dtype=torch.bfloat16, device=device)
    print(f"SERVER: Using {device} device.")

    @batch
    def _infer_fn(**inputs):
        prompts = np.squeeze(inputs["prompts"]).tolist()
        decoded_prompts = [p.decode("utf-8") for p in prompts]
        responses = pipe(decoded_prompts, max_new_tokens=256, temperature=0.2)  # limit responses to 256 tokens, since reasoning tasks can take a while
        return {
            "responses": np.array([r[0]['generated_text'] for r in responses]).reshape(-1, 1)
        }

    workspace_path = f"/tmp/triton_{time.strftime('%m_%d_%M_%S')}"
    triton_conf = TritonConfig(http_port=ports[0], grpc_port=ports[1], metrics_port=ports[2])
    with Triton(config=triton_conf, workspace=workspace_path) as triton:
        triton.bind(
            model_name="deepseek-r1",
            infer_func=_infer_fn,
            inputs=[
                Tensor(name="prompts", dtype=object, shape=(-1,)),
            ],
            outputs=[
                Tensor(name="responses", dtype=object, shape=(-1,)),
            ],
            config=ModelConfig(
                max_batch_size=16,
                batcher=DynamicBatcher(max_queue_delay_microseconds=5000),  # 5ms
            ),
            strict=True,
        )

        def _stop_triton(signum, frame):
            print("SERVER: Received SIGTERM. Stopping Triton server.")
            triton.stop()

        signal.signal(signal.SIGTERM, _stop_triton)

        print("SERVER: Serving inference")
        triton.serve()

#### Start Triton servers

**Specify the number of nodes in the cluster.**  
Following the README, the example standalone cluster uses 1 node. The example Databricks/Dataproc cluster scripts use 4 nodes by default. 

In [21]:
# Change based on cluster setup
num_nodes = 1 if on_standalone else 4

The `TritonClusterManager` will handle the lifecycle of Triton server instances across the Spark cluster:
- Find available ports for HTTP/gRPC/metrics
- Deploy a server on each node via stage-level scheduling
- Gracefully shutdown servers across nodes

In [22]:
model_name = "deepseek-r1"
server_manager = TritonServerManager(num_nodes=num_nodes, model_name=model_name)

In [23]:
# Returns {'hostname', (server_pid, [http_port, grpc_port, metrics_port])}
server_manager.start_servers(triton_server)

2025-02-09 20:43:52,357 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)
2025-02-09 20:43:52,360 - INFO - Starting 1 servers.


{'cb4ae00-lcedt': (4009258, [7000, 7001, 7002])}

#### Define client function

Get the hostname -> url mapping from the server manager:

In [24]:
host_to_grpc_url = server_manager.host_to_grpc_url  # or server_manager.host_to_http_url

Define the Triton inference function, which returns a predict function for batch inference through the server:

In [25]:
def triton_fn(model_name, host_to_url):
    import socket
    import numpy as np
    from pytriton.client import ModelClient

    url = host_to_url[socket.gethostname()]
    print(f"Connecting to Triton model {model_name} at {url}.")

    def infer_batch(inputs):
        with ModelClient(url, model_name, inference_timeout_s=500) as client:
            flattened = np.squeeze(inputs).tolist()
            # Encode batch
            encoded_batch = [[text.encode("utf-8")] for text in flattened]
            encoded_batch_np = np.array(encoded_batch, dtype=np.bytes_)
            # Run inference
            result_data = client.infer_batch(encoded_batch_np)
            result_data = np.squeeze(result_data["responses"], -1)
            return result_data
        
    return infer_batch

In [26]:
generate = predict_batch_udf(partial(triton_fn, model_name=model_name, host_to_url=host_to_grpc_url),
                             return_type=StringType(),
                             input_tensor_shapes=[[1]],
                             batch_size=4)

#### Load and preprocess DataFrame

We'll select a few of the shorter questions for demonstration.

In [ ]:
df = spark.read.json("spark-dl-datasets/orca_math")
df = df.filter(length(col("question")) <= 100).limit(16).repartition(8).cache()

#### Run Inference

In [28]:
%%time
# first pass caches model/fn
preds = df.withColumn("response", generate(col("question")))
results = preds.collect()

CPU times: user 13.5 ms, sys: 6.73 ms, total: 20.2 ms
Wall time: 1min 45s


In [30]:
%%time
preds = df.withColumn("response", generate("question"))
results = preds.collect()

CPU times: user 9.55 ms, sys: 4.51 ms, total: 14.1 ms
Wall time: 1min 45s


In [35]:
print(f"Q: {results[2].question} \n")
print(f"A: {results[2].response} \n")

Q: There are 9 dogs and 23 cats. How many more cats are there than dogs? 

A: There are 9 dogs and 23 cats. How many more cats are there than dogs? Let me think. So, I have 23 cats and 9 dogs. To find out how many more cats there are than dogs, I need to subtract the number of dogs from the number of cats. That would be 23 minus 9. Hmm, 23 minus 9 is 14. So, there are 14 more cats than dogs. Yeah, that makes sense because 23 is a larger number than 9, so subtracting the smaller number from the larger one gives the difference. I don't think I made any mistakes here. It seems straightforward. So, the answer should be 14.
**Final Answer**
The number of cats exceeds the number of dogs by \boxed{14}.
\boxed{14}
</think>

To determine how many more cats there are than dogs, we subtract the number of dogs from the number of cats. 

Given:
- Number of cats = 23
- Number of dogs = 9

The calculation is as follows:
\[ 23 - 9 = 14 \]

Thus, the number of cats exceeds the number of dogs by \boxed{

#### Shut down server on each executor

In [36]:
server_manager.stop_servers()

2025-02-09 20:51:42,370 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)


2025-02-09 20:51:47,588 - INFO - Sucessfully stopped 1 servers.                 


[True]

In [37]:
if not on_databricks: # on databricks, spark.stop() puts the cluster in a bad state
    spark.stop()